# Sentence Boundary detection with NER features

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import torch
import pandas as pd
import numpy as np
import transformers
from tqdm import tqdm
import csv

2022-01-07 07:48:21.799144: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
def set_newline_to_EOS(file_r, file_w, eos="\n"):
    with open(file_w, 'w', encoding="UTF-8") as fw:
        with open(file_r, 'r', encoding="UTF-8") as fr:
            while True:
                line = fr.readline()
                if not line: break
                if line == eos:
                    fw.write("<EOS>\t<EOS>\t<EOS>\t<EOS>\n")
                else:
                    fw.write(line)
                    
def set_sentence_num(df): 

    sent_num = 0
    df['sent_num'] = sent_num
    for idx in range(len(df)):
        df['sent_num'][idx] = sent_num
        #if df[0][idx]=='.' and df[1][idx]=="SENT":
        if df[0][idx]=='<EOS>' and df[1][idx]=="<EOS>":
            sent_num +=1
    df.head()
    df = df[df[0] != "<EOS>"] #get rid of the "<EOS>" toekns
    print(sent_num)
    
    return df


train_file = './data/train.tsv'
test_file = './data/test.tsv'
#test_file = './data_v1/europarl-sbd-eval.tsv'
train_EOS_file = './data/train_EOS.tsv'
test_EOS_file = './data/test_EOS.tsv'
#test_EOS_file = './data_v1/europarl-sbd-eval_EOS.tsv'

set_newline_to_EOS(train_file, train_EOS_file, eos="\n")
set_newline_to_EOS(test_file, test_EOS_file, eos="\n")
#set_newline_to_EOS(test_file, test_EOS_file, eos="\t\n")
train_file = train_EOS_file
test_file = test_EOS_file

In [4]:
train_df = pd.read_csv(train_file, delimiter='\t', engine='python', encoding='UTF-8', error_bad_lines=False, header=None, quoting=csv.QUOTE_NONE)
train_df.head()
test_df = pd.read_csv(test_file, delimiter='\t', engine='python', encoding='UTF-8', error_bad_lines=False, header=None, quoting=csv.QUOTE_NONE)
test_df.head()

,0,1,2,3
0,La,DET:ART,O,B-SENT
1,cérémonie,NOM,O,O
2,aura,VER:futu,O,O
3,lieu,NOM,O,O
4,le,DET:ART,O,O


In [5]:
train_df = set_sentence_num(train_df)
test_df = set_sentence_num(test_df)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


29100
2593


In [6]:
train_df = train_df[train_df['sent_num'] < 1000000]
test_df = test_df[test_df['sent_num'] < 10000]

In [7]:
sentence = train_df[train_df['sent_num']==0]
token_list =  ' '.join([token for token in sentence[0]])
print(token_list)

Les Sables-d'Olonne La Chaume Philippe et Véronique Taillier , son neveu et sa nièce ; Anne Nguyen , Albert et Chantal Merlin , son beau-frère et sa belle-soeur , ont la tristesse de vous faire part du décès de Monsieur Serge SALIS survenu le 26 novembre 2014 , à l' âge de 82 ans .


In [8]:
class FSBDataset():
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.idxtob = {'B-SENT': 1}
        self.idxtoPOS = {'DET:ART': 0,'NAM': 1,'KON': 2,'PUN': 3,'DET:POS': 4,'NOM': 5,'VER:pres': 6,'PRP': 7,'PRO:PER': 8,'VER:infi': 9,'PRP:det': 10,'VER:simp': 11,'VER:pper': 12,'NUM': 13,'SENT': 14,'ABR': 15,'VER:futu': 16,'PRO:DEM': 17,'ADJ': 18,
 'PRO:REL': 19,'PRO:IND': 20,'ADV': 21,'SYM': 22,'PUN:cit': 23,'VER:impf': 24,'VER:subp': 25,'VER:subi': 26,'VER:ppre': 27,'VER:cond': 28,'PRO:POS': 29,'VER:impe': 30}
        self.idxtoNER = {'I-LOC': 0, 'I-PER': 1, 'O': 2, 'I-ORG': 3}
        
    def __len__(self):
        return self.data['sent_num'].max()
    
    def __getitem__(self, item):
        
        sentence = self.data[self.data['sent_num']==item]
        token_list =  [token for token in sentence[0]]
        sbd_list =  [target for target in sentence[3]]
        pos_list =  [target for target in sentence[1]]
        ner_list =  [target for target in sentence[2]]
        
        #target_ids_list =  [1 if token=="B-SENT" else 0 for token in sentence[3]]
        #pos_ids = [self.idxtoPOS.get(pos) for pos in sentence[1]]
        #ner_ids = [self.idxtoNER.get(ner) for ner in sentence[2]]
        
        encoded = self.tokenizer.encode_plus(' '.join(token_list),
                                            None,
                                            add_special_tokens=True,
                                            max_length=self.max_length,
                                            truncation=True,
                                            padding='max_length')
        
        ids = encoded['input_ids']
        mask = encoded['attention_mask']
        
        bpe_head_mask = [0]; sbd_ids = [-1]; pos_ids = [-1]; ner_ids = [-1] # --> CLS token
        
        for word, sbd, pos, ner in zip(token_list, sbd_list, pos_list, ner_list):
            bpe_len = len(self.tokenizer.tokenize(word))
            head_mask = [1] + [0]*(bpe_len-1)
            bpe_head_mask.extend(head_mask)
            
            sbd_mask = [self.idxtob.get(sbd,0)] + [-1]*(bpe_len-1)
            sbd_ids.extend(sbd_mask)
            pos_mask = [self.idxtoPOS.get(pos,0)] + [-1]*(bpe_len-1)
            pos_ids.extend(pos_mask)
            ner_mask = [self.idxtoNER.get(ner,0)] + [-1]*(bpe_len-1)
            ner_ids.extend(ner_mask)
            #print("head_mask", head_mask)
        
        bpe_head_mask.append(0)
        bpe_head_mask.extend([0] * (self.max_length - len(bpe_head_mask)))
        
        sbd_ids.append(-1) # --> END token
        sbd_ids.extend([-1] * (self.max_length - len(sbd_ids))) ## --> padding by max_len
        pos_ids.append(-1) # --> END token
        pos_ids.extend([-1] * (self.max_length - len(pos_ids))) ## --> padding by max_len
        ner_ids.append(-1) # --> END token
        ner_ids.extend([-1] * (self.max_length - len(ner_ids))) ## --> padding by max_len


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'bpe_head_mask': torch.tensor(bpe_head_mask, dtype=torch.long),
            'sbd_ids': torch.tensor(sbd_ids, dtype=torch.long),
            'pos_ids': torch.tensor(pos_ids, dtype=torch.long),
            'ner_ids': torch.tensor(ner_ids, dtype=torch.long)
        }
        
        
        
        

In [9]:
class XLMRobertaBaseline(torch.nn.Module):
    def __init__(self):
        super(XLMRobertaBaseline, self).__init__()
        
        self.bert = transformers.XLMRobertaModel.from_pretrained('xlm-roberta-base')
        self.dropout = torch.nn.Dropout(0.33)
        self.classfier = torch.nn.Linear(768, 2)
        
        
    def forward(self, ids, mask):
        
        o1, o2 = self.bert(ids, mask)
        out = self.dropout(o1)
        logits = self.classfier(out)
        
        return logits
        
        
    
class POS(torch.nn.Module):
    def __init__(self, num_pos=31):
        super(POS, self).__init__()
        
        self.dropout = torch.nn.Dropout(0.33)
        self.classfier = torch.nn.Linear(768, num_pos)
        
    def forward(self, bert_out):
        
        o1 = bert_out
        out = self.dropout(o1)
        logits = self.classfier(out)
        
        return logits
    
    
class NER(torch.nn.Module):
    def __init__(self, num_ner=4):
        super(NER, self).__init__()
        
        self.dropout = torch.nn.Dropout(0.33)
        self.classfier = torch.nn.Linear(768, num_ner)
        
    def forward(self, bert_out):
        
        o1 = bert_out
        out = self.dropout(o1)
        logits = self.classfier(out)
        
        return logits
    
    
class SBD(torch.nn.Module):
    def __init__(self, input_dim=768, num_sbd=2):
        super(SBD, self).__init__()
        
        self.dropout = torch.nn.Dropout(0.33)
        self.classfier = torch.nn.Linear(input_dim, num_sbd)
        
    def forward(self, bert_out):
        
        o1 = bert_out
        out = self.dropout(o1)
        logits = self.classfier(out)
        
        return logits
    
    
class XLMRobertaMultiTask(torch.nn.Module):
    def __init__(self):
        super(XLMRobertaMultiTask, self).__init__()
        
        num_pos = 31
        num_ner = 4
        num_sbd = 2
        pos_dim = 50
        ner_dim = 25
        
        self.bert = transformers.XLMRobertaModel.from_pretrained('xlm-roberta-base')
        
        self.pos = POS(num_pos)
        self.ner = NER(num_ner)
        self.sbd = SBD(num_sbd)
        
        self.pos_emb = torch.nn.Embedding(num_pos, pos_dim)
        self.ner_emb = torch.nn.Embedding(num_ner, ner_dim)
        
        
    def forward(self, ids, mask):
        
        o1, o2 = self.bert(ids, mask)
        out = o1 
        
        pos_logits = self.pos(out)
        ner_logits = self.ner(out)
        sbd_logits = self.sbd(out)

        
        return sbd_logits, pos_logits, ner_logits
    
    
    
class XLMRobertaMultiTaskPred(torch.nn.Module):
    def __init__(self):
        super(XLMRobertaMultiTaskPred, self).__init__()
        
        num_pos = 31
        num_ner = 4
        num_sbd = 2
        pos_dim = 50
        ner_dim = 25
        
        self.bert = transformers.XLMRobertaModel.from_pretrained('xlm-roberta-base')
        
        self.pos = POS(num_pos)
        self.ner = NER(num_ner)
        self.sbd = SBD(768+pos_dim+ner_dim, num_sbd)
        
        self.pos_emb = torch.nn.Embedding(num_pos, pos_dim)
        self.ner_emb = torch.nn.Embedding(num_ner, ner_dim)
        
        
    def forward(self, ids, mask):
        
        o1, o2 = self.bert(ids, mask)
        out = o1 #self.dropout(o1)
        
        #Step1: predict POS tags for the entire toekns
        pos_logits = self.pos(out)
        ner_logits = self.ner(out)
        
        pos_idx = torch.argmax(pos_logits, dim=2)
        ner_idx = torch.argmax(ner_logits, dim=2)
        
        pos_emb = self.pos_emb(pos_idx)
        ner_emb = self.ner_emb(ner_idx)

        concatenated = torch.cat([out, pos_emb, ner_emb], dim=2)
        sbd_logits = self.sbd(concatenated)
        
        return sbd_logits, pos_logits, ner_logits

In [10]:
MAX_LEN = 510
tokenizer = transformers.XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
train_dataset = FSBDataset(train_df, tokenizer, MAX_LEN)
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, num_workers=4, batch_size=8)
test_dataset = FSBDataset(test_df, tokenizer, MAX_LEN)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, num_workers=4, batch_size=6)

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

In [11]:
#model = XLMRobertaBaseline()
#model = XLMRobertaMultiTask()
model = XLMRobertaMultiTaskPred()
model = torch.nn.DataParallel(model)
model = model.cuda()
optimizer = transformers.AdamW(params=model.parameters(), lr=0.000005)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-1)

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [12]:
def f1_score(total_pred, total_targ):
    
    p = 0 # (retrived SB and real SB) / retrived SB  # The percentage of (the number of correct predictions) / (the number of predction that system predicts as B-SENT)
    r = 0
    f1= 0

    np_total_pred = np.array(total_pred)
    np_total_tag = np.array(total_targ)
    
    #precision
    incidence_nopad = np.where(np_total_tag != -1) ## eliminate paddings
    #print("incidence_nopad", incidence_nopad)
    
    np_total_pred_nopad = np_total_pred[incidence_nopad]
    np_total_tag_nopad = np_total_tag[incidence_nopad]
    
    incidence_nopad_sb = np.where(np_total_pred_nopad == 1)
    np_total_pred_nopad_sb = np_total_pred_nopad[incidence_nopad_sb]
    np_total_tag_nopad_sb = np_total_tag_nopad[incidence_nopad_sb]
    
    count_active_tokens_p = len(np_total_pred_nopad_sb)
    count_correct_p = np.count_nonzero((np_total_pred_nopad_sb==np_total_tag_nopad_sb) == True)
    
    print("count_correct_p", count_correct_p)
    print("count_active_tokens_p", count_active_tokens_p)
    
    
    try:
        p = count_correct_p/count_active_tokens_p
    except ZeroDivisionError:
        p = 0
    

    print("precision:", p)

    
    #recall
    ids_sb_pred_r = np.where(np_total_tag==1)
    np_total_pred_r = np_total_pred[ids_sb_pred_r]
    np_total_tag_r = np_total_tag[ids_sb_pred_r]
    
    count_active_tokens_r = len(np_total_pred_r)
    count_correct_r = np.count_nonzero((np_total_pred_r==np_total_tag_r) == True)
    
    print("count_active_tokens_r", count_active_tokens_r)
    print("count_correct_r", count_correct_r)
    
    
    
    try:
        r = count_correct_r/count_active_tokens_r
    except ZeroDivisionError:
        r = 0
    
    print("recall:", r)
    
    
    try:
        f1 = 2*(p*r) / (p+r)
    except ZeroDivisionError:
        f1 = 0
    

    print("F1:", f1)
    
    
    
def ner_f1_score(total_pred, total_targ):
    
    p = 0 # (retrived SB and real SB) / retrived SB  # The percentage of (the number of correct predictions) / (the number of predction that system predicts as B-SENT)
    r = 0
    f1= 0

    np_total_pred = np.array(total_pred)
    np_total_tag = np.array(total_targ)
    
    #precision
    incidence_nopad = np.where(np_total_tag != -1) ## eliminate paddings
    #print("incidence_nopad", incidence_nopad)
    
    np_total_pred_nopad = np_total_pred[incidence_nopad]
    np_total_tag_nopad = np_total_tag[incidence_nopad]
    
    incidence_nopad_sb = np.where((np_total_pred_nopad != -1) & (np_total_pred_nopad != 2)) #np.where(np_total_pred_nopad == 1) 
    np_total_pred_nopad_sb = np_total_pred_nopad[incidence_nopad_sb]
    np_total_tag_nopad_sb = np_total_tag_nopad[incidence_nopad_sb]
    
    count_active_tokens_p = len(np_total_pred_nopad_sb)
    count_correct_p = np.count_nonzero((np_total_pred_nopad_sb==np_total_tag_nopad_sb) == True)
    
    print("count_correct_p", count_correct_p)
    print("count_active_tokens_p", count_active_tokens_p)
    
    
    try:
        p = count_correct_p/count_active_tokens_p
    except ZeroDivisionError:
        p = 0
    

    print("precision:", p)

    
    #recall
    ids_sb_pred_r = np.where((np_total_tag != -1) & (np_total_tag != 2)) #np.where(np_total_tag==1)
    np_total_pred_r = np_total_pred[ids_sb_pred_r]
    np_total_tag_r = np_total_tag[ids_sb_pred_r]
    
    count_active_tokens_r = len(np_total_pred_r)
    count_correct_r = np.count_nonzero((np_total_pred_r==np_total_tag_r) == True)
    
    print("count_active_tokens_r", count_active_tokens_r)
    print("count_correct_r", count_correct_r)
    
    
    
    try:
        r = count_correct_r/count_active_tokens_r
    except ZeroDivisionError:
        r = 0
    
    print("recall:", r)
    
    
    try:
        f1 = 2*(p*r) / (p+r)
    except ZeroDivisionError:
        f1 = 0
    

    print("F1:", f1)
    

In [13]:
def train_loop_fn(train_loader, model, optimizer, DEVICE=None, scheduler=None, mode="all", s_r=1, p_r=1, n_r=1):
    model.train()
    
    sbd_pred = []
    sbd_targ = []
    sbd_loss = []
    
    pos_pred = []
    pos_targ = []
    pos_loss = []
    
    ner_pred = []
    ner_targ = []
    ner_loss = []
    
    for idx, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
        optimizer.zero_grad()

        sbd_logits, pos_logits, ner_logits = model(batch['ids'].cuda(), batch['mask'].cuda())
        
        #if mode = "all" or "sbd":
        b,s,l = sbd_logits.size()
        s_loss = loss_fn(sbd_logits.view(b*s,l), batch['sbd_ids'].cuda().view(b*s))
        sbd_loss.append(s_loss.item())
        sbd_pred.extend(torch.argmax(sbd_logits.view(b*s,l), 1).cpu().tolist())
        sbd_targ.extend(batch['sbd_ids'].cuda().view(b*s).cpu().tolist())

        b,s,l = pos_logits.size()
        p_loss = loss_fn(pos_logits.view(b*s,l), batch['pos_ids'].cuda().view(b*s))
        pos_loss.append(p_loss.item())
        pos_pred.extend(torch.argmax(pos_logits.view(b*s,l), 1).cpu().tolist())
        pos_targ.extend(batch['pos_ids'].cuda().view(b*s).cpu().tolist())
        
        b,s,l = ner_logits.size()
        n_loss = loss_fn(ner_logits.view(b*s,l), batch['ner_ids'].cuda().view(b*s))
        ner_loss.append(n_loss.item())
        ner_pred.extend(torch.argmax(ner_logits.view(b*s,l), 1).cpu().tolist())
        ner_targ.extend(batch['ner_ids'].cuda().view(b*s).cpu().tolist())
        
        loss = s_r*s_loss + n_r*n_loss + p_r*p_loss
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
            
    sbd_count_active_tokens = np.count_nonzero(np.array(sbd_targ) > -1)
    sbd_count_correct = np.count_nonzero((np.array(sbd_pred)==np.array(sbd_targ)) == True)
    pos_count_active_tokens = np.count_nonzero(np.array(pos_targ) > -1)
    pos_count_correct = np.count_nonzero((np.array(pos_pred)==np.array(pos_targ)) == True)
    ner_count_active_tokens = np.count_nonzero(np.array(ner_targ) > -1)
    ner_count_correct = np.count_nonzero((np.array(ner_pred)==np.array(ner_targ)) == True)
    
    f1_score(sbd_pred, sbd_targ)
    print("POS Accuracy:"+ str(pos_count_correct/pos_count_active_tokens))
    print("NER Accuracy:"+ str(ner_count_correct/ner_count_active_tokens))
    ner_f1_score(ner_pred, ner_targ)
    
    #f1_score(total_pred[2:], total_targ[2:])

    
def dev_loop_fn(dev_loader, model, optimizer, DEVICE=None, scheduler=None):
    model.eval()
    
    sbd_pred = []
    sbd_targ = []
    sbd_loss = []
    sbd_middle_pred = []
    sbd_middle_targ = []
    
    pos_pred = []
    pos_targ = []
    pos_loss = []
    
    ner_pred = []
    ner_targ = []
    ner_loss = []


    with torch.no_grad():
        for idx, batch in tqdm(enumerate(dev_loader), total=len(dev_loader)):

            sbd_logits, pos_logits, ner_logits = model(batch['ids'].cuda(), batch['mask'].cuda())

            #if mode = "all" or "sbd":
            b,s,l = sbd_logits.size()
            s_loss = loss_fn(sbd_logits.view(b*s,l), batch['sbd_ids'].cuda().view(b*s))
            sbd_loss.append(s_loss.item())
            sbd_pred.extend(torch.argmax(sbd_logits.view(b*s,l), 1).cpu().tolist())
            sbd_targ.extend(batch['sbd_ids'].cuda().view(b*s).cpu().tolist())

            b,s,l = pos_logits.size()
            p_loss = loss_fn(pos_logits.view(b*s,l), batch['pos_ids'].cuda().view(b*s))
            pos_loss.append(p_loss.item())
            pos_pred.extend(torch.argmax(pos_logits.view(b*s,l), 1).cpu().tolist())
            pos_targ.extend(batch['pos_ids'].cuda().view(b*s).cpu().tolist())

            b,s,l = ner_logits.size()
            n_loss = loss_fn(ner_logits.view(b*s,l), batch['ner_ids'].cuda().view(b*s))
            ner_loss.append(n_loss.item())
            ner_pred.extend(torch.argmax(ner_logits.view(b*s,l), 1).cpu().tolist())
            ner_targ.extend(batch['ner_ids'].cuda().view(b*s).cpu().tolist())
            

            sbd_logits_middle = sbd_logits[:,2:,]
            b,s,l = sbd_logits_middle.size()
            #print(b,s,l)
            sbd_middle_pred.extend(torch.argmax(sbd_logits_middle.contiguous().view(b*s,l), 1).cpu().tolist())
            sbd_middle_targ.extend(batch['sbd_ids'][:,2:].cuda().contiguous().view(b*s).cpu().tolist())
            

    sbd_count_active_tokens = np.count_nonzero(np.array(sbd_targ) > -1)
    sbd_count_correct = np.count_nonzero((np.array(sbd_pred)==np.array(sbd_targ)) == True)
    pos_count_active_tokens = np.count_nonzero(np.array(pos_targ) > -1)
    pos_count_correct = np.count_nonzero((np.array(pos_pred)==np.array(pos_targ)) == True)
    ner_count_active_tokens = np.count_nonzero(np.array(ner_targ) > -1)
    ner_count_correct = np.count_nonzero((np.array(ner_pred)==np.array(ner_targ)) == True)

    f1_score(sbd_pred, sbd_targ)
    f1_score(sbd_middle_pred, sbd_middle_targ)
    print("POS Accuracy:"+ str(pos_count_correct/pos_count_active_tokens))
    print("NER Accuracy:"+ str(ner_count_correct/ner_count_active_tokens))

    '''
    print("sbd_targ: ",sbd_targ)
    print("sbd_pred: ",sbd_pred)

    print("pos_targ: ",pos_targ)
    print("pos_pred: ",pos_pred)

    print("ner_targ: ",ner_targ)
    print("ner_pred: ",ner_pred)
    '''
    ner_f1_score(ner_pred, ner_targ)


    
    
    #count_active_tokens = np.count_nonzero(np.array(total_targ) > -1)
    #count_correct = np.count_nonzero((np.array(total_pred)==np.array(total_targ)) == True)
    #print("TESTING ACC:", count_correct/count_active_tokens)
    

In [14]:
for idx in range(100):
    train_loop_fn(train_loader, model, optimizer, s_r=1, p_r=1, n_r=1)
    dev_loop_fn(test_loader, model, optimizer)

'''
for idx in range(20):
    train_loop_fn(train_loader, model, optimizer, a=0,b=0,c=1)
    dev_loop_fn(test_loader, model, optimizer)
print("Finished NER training")
    
for idx in range(20):
    train_loop_fn(train_loader, model, optimizer, a=0,b=1,c=0)
    dev_loop_fn(test_loader, model, optimizer)
print("Finished POS training")
    
for idx in range(20):
    train_loop_fn(train_loader, model, optimizer, a=1,b=0,c=0)
    dev_loop_fn(test_loader, model, optimizer)
print("Finished SBD training")
'''

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3638/3638 [11:04<00:00,  5.48it/s]


count_correct_p 29110
count_active_tokens_p 30839
precision: 0.9439346282304873
count_active_tokens_r 33627
count_correct_r 29110
recall: 0.8656734171945163
F1: 0.9031117178047343
POS Accuracy:0.9409833444300566
NER Accuracy:0.9671751062926285
count_correct_p 109764
count_active_tokens_p 122709
precision: 0.8945065154145173
count_active_tokens_r 128847
count_correct_r 109764
recall: 0.8518941069640736
F1: 0.8726804369603588


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 432/432 [00:18<00:00, 23.11it/s]


count_correct_p 3100
count_active_tokens_p 3257
precision: 0.9517961314092723
count_active_tokens_r 3146
count_correct_r 3100
recall: 0.9853782581055308
F1: 0.968296111197876
count_correct_p 508
count_active_tokens_p 665
precision: 0.7639097744360902
count_active_tokens_r 554
count_correct_r 508
recall: 0.9169675090252708
F1: 0.8334700574241182
POS Accuracy:0.98529204122935
NER Accuracy:0.9797971478326352
count_correct_p 14028
count_active_tokens_p 15358
precision: 0.9134001823154057
count_active_tokens_r 15103
count_correct_r 14028
recall: 0.9288220883268224
F1: 0.9210465841567905


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3638/3638 [11:10<00:00,  5.43it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3638/3638 [20:06<00:00,  3.01it/s]


count_correct_p 33612
count_active_tokens_p 33630
precision: 0.9994647636039251
count_active_tokens_r 33627
count_correct_r 33612
recall: 0.9995539298777768
F1: 0.9995093447522192
POS Accuracy:0.9998689363397589
NER Accuracy:0.9997929194168191
count_correct_p 128739
count_active_tokens_p 128852
precision: 0.9991230248657375
count_active_tokens_r 128847
count_correct_r 128739
recall: 0.9991617965493957
F1: 0.9991424103314333


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 432/432 [00:27<00:00, 15.58it/s]


count_correct_p 3098
count_active_tokens_p 3183
precision: 0.9732956330505812
count_active_tokens_r 3146
count_correct_r 3098
recall: 0.9847425301970757
F1: 0.9789856217411913
count_correct_p 506
count_active_tokens_p 591
precision: 0.856175972927242
count_active_tokens_r 554
count_correct_r 506
recall: 0.9133574007220217
F1: 0.8838427947598254
POS Accuracy:0.9920930013648985
NER Accuracy:0.9869275662446463
count_correct_p 14339
count_active_tokens_p 15179
precision: 0.9446603860596877
count_active_tokens_r 15103
count_correct_r 14339
recall: 0.9494140237038999
F1: 0.9470312396803381


'\nfor idx in range(20):\n    train_loop_fn(train_loader, model, optimizer, a=0,b=0,c=1)\n    dev_loop_fn(test_loader, model, optimizer)\nprint("Finished NER training")\n    \nfor idx in range(20):\n    train_loop_fn(train_loader, model, optimizer, a=0,b=1,c=0)\n    dev_loop_fn(test_loader, model, optimizer)\nprint("Finished POS training")\n    \nfor idx in range(20):\n    train_loop_fn(train_loader, model, optimizer, a=1,b=0,c=0)\n    dev_loop_fn(test_loader, model, optimizer)\nprint("Finished SBD training")\n'

# Start fine-tuning and test on Europal data
 - spliting Europal data as the train and test split with 80:20 ratio
 - From now this on the model is trained on top of the pretrained model

In [15]:
train_file = './data/train.tsv'
test_file = './data/test.tsv'
test_file = './data_v1/europarl-sbd-eval.tsv'
train_EOS_file = './data/train_EOS.tsv'
test_EOS_file = './data/test_EOS.tsv'
test_EOS_file = './data_v1/europarl-sbd-eval_EOS.tsv'

set_newline_to_EOS(train_file, train_EOS_file, eos="\n")
set_newline_to_EOS(test_file, test_EOS_file, eos="\n")
set_newline_to_EOS(test_file, test_EOS_file, eos="\t\n")
train_file = train_EOS_file
test_file = test_EOS_file

In [16]:
train_df = pd.read_csv(train_file, delimiter='\t', engine='python', encoding='UTF-8', error_bad_lines=False, header=None, quoting=csv.QUOTE_NONE)
train_df.head()

,0,1,2,3
0,Les,DET:ART,I-LOC,B-SENT
1,Sables-d'Olonne,NAM,I-LOC,O
2,La,DET:ART,I-LOC,O
3,Chaume,NAM,I-LOC,O
4,Philippe,NAM,I-PER,B-SENT


In [17]:
test_df = pd.read_csv(test_file, delimiter='\t', engine='python', encoding='UTF-8', error_bad_lines=False, header=None, quoting=csv.QUOTE_NONE)
test_df.head()

,0,1,2,3
0,Reprise,NOM,O,B-SENT
1,de,PRP,O,O
2,la,DET:ART,O,O
3,session,NOM,O,O
4,Je,PRO:PER,O,B-SENT


In [18]:
train_df = set_sentence_num(train_df)
test_df = set_sentence_num(test_df)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


29100
49709


In [19]:
train_df = test_df[test_df['sent_num'] >= 9942]
test_df = test_df[test_df['sent_num'] <= 9941]

In [20]:
train_dataset = FSBDataset(train_df, tokenizer, MAX_LEN)
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, num_workers=4, batch_size=8)
test_dataset = FSBDataset(test_df, tokenizer, MAX_LEN)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, num_workers=4, batch_size=6)

In [21]:
for idx in range(10):
    train_loop_fn(train_loader, model, optimizer, s_r=1, p_r=1, n_r=1)
    dev_loop_fn(test_loader, model, optimizer)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6214/6214 [20:30<00:00,  5.05it/s]


count_correct_p 39884
count_active_tokens_p 39918
precision: 0.9991482539205371
count_active_tokens_r 40103
count_correct_r 39884
recall: 0.9945390619155674
F1: 0.9968383299383913
POS Accuracy:0.9831000033355256
NER Accuracy:0.9949368431045704
count_correct_p 14138
count_active_tokens_p 16944
precision: 0.834395656279509
count_active_tokens_r 18220
count_correct_r 14138
recall: 0.77596048298573
F1: 0.8041178477988853


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1657/1657 [01:30<00:00, 18.26it/s]


count_correct_p 9988
count_active_tokens_p 9989
precision: 0.9998998898788668
count_active_tokens_r 10025
count_correct_r 9988
recall: 0.9963092269326683
F1: 0.9981013290696513
count_correct_p 47
count_active_tokens_p 48
precision: 0.9791666666666666
count_active_tokens_r 84
count_correct_r 47
recall: 0.5595238095238095
F1: 0.712121212121212
POS Accuracy:0.988266811937993
NER Accuracy:0.9958487339328074
count_correct_p 3491
count_active_tokens_p 4169
precision: 0.8373710721995682
count_active_tokens_r 4268
count_correct_r 3491
recall: 0.8179475164011246
F1: 0.8275453360199121


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6214/6214 [25:10<00:00,  4.11it/s]


...

count_correct_p 9992
count_active_tokens_p 9998
precision: 0.9993998799759952
count_active_tokens_r 10025
count_correct_r 9992
recall: 0.9967082294264339
F1: 0.9980522399240873
count_correct_p 51
count_active_tokens_p 57
precision: 0.8947368421052632
count_active_tokens_r 84
count_correct_r 51
recall: 0.6071428571428571
F1: 0.7234042553191489
POS Accuracy:0.9923491200970928
NER Accuracy:0.9966900204115408
count_correct_p 3674
count_active_tokens_p 4272
precision: 0.8600187265917603
count_active_tokens_r 4268
count_correct_r 3674
recall: 0.8608247422680413
F1: 0.8604215456674473


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6214/6214 [25:12<00:00,  4.11it/s]


count_correct_p 40041
count_active_tokens_p 40065
precision: 0.9994009734181954
count_active_tokens_r 40103
count_correct_r 40041
recall: 0.9984539809989278
F1: 0.9989272527691847
POS Accuracy:0.9973495399968013
NER Accuracy:0.9988000660263028
count_correct_p 17311
count_active_tokens_p 18047
precision: 0.9592175984928243
count_active_tokens_r 18220
count_correct_r 17311
recall: 0.9501097694840834
F1: 0.9546419610113877


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1657/1657 [01:28<00:00, 18.72it/s]


count_correct_p 9991
count_active_tokens_p 9996
precision: 0.999499799919968
count_active_tokens_r 10025
count_correct_r 9991
recall: 0.9966084788029925
F1: 0.9980520453523799
count_correct_p 50
count_active_tokens_p 55
precision: 0.9090909090909091
count_active_tokens_r 84
count_correct_r 50
recall: 0.5952380952380952
F1: 0.7194244604316546
POS Accuracy:0.9923939427373532
NER Accuracy:0.9967831135874662
count_correct_p 3738
count_active_tokens_p 4360
precision: 0.8573394495412844
count_active_tokens_r 4268
count_correct_r 3738
recall: 0.8758200562324273
F1: 0.866481223922114


In [24]:
train_file = './data/train.tsv'
test_file = './data/test.tsv'
#test_file = './data_v1/europarl-sbd-eval.tsv'
train_EOS_file = './data/train_EOS.tsv'
test_EOS_file = './data/test_EOS.tsv'
#test_EOS_file = './data_v1/europarl-sbd-eval_EOS.tsv'

set_newline_to_EOS(train_file, train_EOS_file, eos="\n")
set_newline_to_EOS(test_file, test_EOS_file, eos="\n")
#set_newline_to_EOS(test_file, test_EOS_file, eos="\t\n")
train_file = train_EOS_file
test_file = test_EOS_file

train_df = pd.read_csv(train_file, delimiter='\t', engine='python', encoding='UTF-8', error_bad_lines=False, header=None, quoting=csv.QUOTE_NONE)
train_df.head()
test_df = pd.read_csv(test_file, delimiter='\t', engine='python', encoding='UTF-8', error_bad_lines=False, header=None, quoting=csv.QUOTE_NONE)
test_df.head()

train_df = set_sentence_num(train_df)
test_df = set_sentence_num(test_df)

train_dataset = FSBDataset(train_df, tokenizer, MAX_LEN)
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, num_workers=4, batch_size=8)
test_dataset = FSBDataset(test_df, tokenizer, MAX_LEN)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, num_workers=4, batch_size=6)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


29100
2593


In [25]:
dev_loop_fn(test_loader, model, optimizer)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 432/432 [00:17<00:00, 24.14it/s]


count_correct_p 3070
count_active_tokens_p 3208
precision: 0.9569825436408977
count_active_tokens_r 3146
count_correct_r 3070
recall: 0.9758423394787031
F1: 0.966320428076802
count_correct_p 479
count_active_tokens_p 617
precision: 0.7763371150729336
count_active_tokens_r 554
count_correct_r 479
recall: 0.8646209386281588
F1: 0.8181041844577285
POS Accuracy:0.991422318444957
NER Accuracy:0.9795029886572222
count_correct_p 13724
count_active_tokens_p 14700
precision: 0.9336054421768708
count_active_tokens_r 15103
count_correct_r 13724
recall: 0.9086936370257565
F1: 0.9209811092843002


In [26]:
for idx in range(10):
    train_loop_fn(train_loader, model, optimizer, s_r=1, p_r=1, n_r=1)
    dev_loop_fn(test_loader, model, optimizer)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3638/3638 [11:05<00:00,  5.47it/s]


count_correct_p 33582
count_active_tokens_p 33625
precision: 0.998721189591078
count_active_tokens_r 33627
count_correct_r 33582
recall: 0.9986617896333304
F1: 0.9986914887289597
POS Accuracy:0.9992883243248911
NER Accuracy:0.9990497884632523
count_correct_p 128341
count_active_tokens_p 128801
precision: 0.9964285991568388
count_active_tokens_r 128847
count_correct_r 128341
recall: 0.9960728616110581
F1: 0.9962506986275849


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 432/432 [00:18<00:00, 23.17it/s]


count_correct_p 3110
count_active_tokens_p 3209
precision: 0.9691492676846369
count_active_tokens_r 3146
count_correct_r 3110
recall: 0.9885568976478067
F1: 0.978756884343037
count_correct_p 518
count_active_tokens_p 617
precision: 0.839546191247974
count_active_tokens_r 554
count_correct_r 518
recall: 0.9350180505415162
F1: 0.8847139197267292
POS Accuracy:0.9931519743963854
NER Accuracy:0.9862451169576881
count_correct_p 14331
count_active_tokens_p 15237
precision: 0.9405394762748572
count_active_tokens_r 15103
count_correct_r 14331
recall: 0.9488843276170297
F1: 0.9446934739617665


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3638/3638 [11:11<00:00,  5.42it/s]


...

count_correct_p 3100
count_active_tokens_p 3179
precision: 0.9751494180559924
count_active_tokens_r 3146
count_correct_r 3100
recall: 0.9853782581055308
F1: 0.9802371541501976
count_correct_p 508
count_active_tokens_p 587
precision: 0.8654173764906303
count_active_tokens_r 554
count_correct_r 508
recall: 0.9169675090252708
F1: 0.8904469763365469
POS Accuracy:0.9932225725984845
NER Accuracy:0.9867981362074646
count_correct_p 14363
count_active_tokens_p 15250
precision: 0.9418360655737705
count_active_tokens_r 15103
count_correct_r 14363
recall: 0.9510031119645104
F1: 0.9463973907027312


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3638/3638 [14:51<00:00,  4.08it/s]


count_correct_p 33613
count_active_tokens_p 33627
precision: 0.999583667885925
count_active_tokens_r 33627
count_correct_r 33613
recall: 0.999583667885925
F1: 0.999583667885925
POS Accuracy:0.9998676257031566
NER Accuracy:0.9997929194168191
count_correct_p 128736
count_active_tokens_p 128858
precision: 0.9990532213754676
count_active_tokens_r 128847
count_correct_r 128736
recall: 0.9991385131202123
F1: 0.9990958654275237


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 432/432 [00:22<00:00, 19.19it/s]


count_correct_p 3093
count_active_tokens_p 3176
precision: 0.9738664987405542
count_active_tokens_r 3146
count_correct_r 3093
recall: 0.9831532104259377
F1: 0.9784878203100285
count_correct_p 502
count_active_tokens_p 585
precision: 0.8581196581196581
count_active_tokens_r 554
count_correct_r 502
recall: 0.9061371841155235
F1: 0.8814749780509218
POS Accuracy:0.9929872452581541
NER Accuracy:0.9868099025744811
count_correct_p 14326
count_active_tokens_p 15159
precision: 0.9450491457220134
count_active_tokens_r 15103
count_correct_r 14326
recall: 0.9485532675627358
F1: 0.9467979644438571


In [ ]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

In [ ]:
checkpoint = {
    "state_dict": model.state_dict(),
    "optimizer": optimizer.state_dict(),
    "train_dataset": train_dataset
}
save_checkpoint(checkpoint)